列生成

Benders分解

用来处理当问题不可解的问题

## 列生成(Column Generation)

### Reduced Cost

$min\ c^T x$

$s.t. Ax = b$

$\ \ \ \ \ \ \  x \ge 0$

令 $x = [x_B, x_N]$, 其中$x_B$表示基变量, $x_N$表示非基变量。相应的$A = [B, N]$, $c^T = [c_B^T, c_N^T]$

限制条件:

$Ax=b \ \leftrightarrow \  Bx_B + Nx_N = b \ \leftrightarrow \ x_B = B^{-1}b - B^{-1}Nx_N$ (非基变量$x_N=0$, $x_B=B^{-1}b$)

目标函数:

$min \ c^T x \ \leftrightarrow \ min \ c^T_B x_B + c^T_N x_N  \ \leftrightarrow \ min\  c^T_B(B^{-1}b - B^{-1} N x_N) + c^T_N x_N  \ \leftrightarrow \ min\  c^T_B B^{-1} b + (c^T_N -c^T_B B^{-1} N) x_N $

- 可以发现 
    - $c^T B^{-1} b$ 是一个常数
    - $(c^T_N -c^T_B B^{-1} N)$ 是一个系数, 我们称之为Reduced Cost
    - $x_N$是一个变量
- 因为目标函数是求最小化
    - $\exists n \in N$, 使得 $c^T_N -c^T_B B^{-1} n \lt 0$, 因此可以适当增加非基变量$x_n$的值，从而降低目标值
    - $\forall n \in N$, 都有 $c^T_N -c^T_B B^{-1} n \ge  0$, 获得最优解

In [2]:
# 其实可以这么理解，非基变量就是额外往模型里增加的变量
# 从实际案例里出发，就是只要Reduced Cost为负值，我们仍可以找到一组新的非基变量增加进去，使得目标函数更小
# 而当Reduced Cost为正值的时候，就没有必要再往模型里增加新的非基变量，当前求出的就已经为最优解了

### Reduced cost and Dual Variables(对偶变量)

$min\ c^T x$

$s.t. Ax = b$

$\ \ \ \ \ \ \  x \ge 0$

DUAL $\leftrightarrow$ DUAL

$max \ y^T b$

$s.t. y^T A \le c^T$

假设找到了原问题的最优解$[x_B,0]$ $\leftrightarrow$ $\forall N$, $c^T_N -c^T_B B^{-1} n \ge  0$, 即 $c^T_B  B^{-1} N \le c^T_N$

**$c^T_B B^{-1}$**$A$ = **$c^T_B B^{-1}$**$[B,N]$ = $[c^T_B,$ **$c^T_B B^{-1}$** $N]$ $\le [c^T_B, c^T_N] = $ **$c^T$** $\leftrightarrow c^T_B B^{-1}A \le c^T \leftrightarrow c^T_B B^{-1}$ 是对偶问题的可行解

令: $y^T = c^T_B B^{-1}$, $y^T b = c^T_B B^{-1} b = c^T_B x_B c^T x$, 根据对偶定理, $y^T$为对偶问题的最优解

- Reduced Cost = $c^T_N -$ Dual Variables*N            (Reduced Cost与对偶变量之间的关系)

### Column Generation 思路

- obj
    - $min \ c_1 x_1 + c_2 x_2 + ...$
- s.t.
    - $a_{11} x_1 + a_{12} x_2 + ... \le b_1$
    - $a_{21} x_1 + a_{22} x_2 + ... \le b_2$
    - ...
    - $a_{m1} x_1 + a_{22} x_2 + ... \le b_2$
    - $x_1, x_2, ... Integer$

对于一个整数规划模型，如果问题变量过多(**往往很难显性枚举出所有变量**), 可以先枚举部分变量构造一个规模较小的模型，然后通过迭代不停向较小模型里添加新的变量(列), 直到没有新的变量(列)添加为止。

**案例**

假设需要长度为3, 7, 9, 16米的钢管各25, 30, 14, 8根， 目前只有长度为20米的钢管若干，合理安排切割方案，使得消耗的钢管数量最少。


**常规思路**
- 参数
    - $I$   -   所需钢管种类集合;
    - $K$   -   目前未切割的钢管集合;
    - $D_i$ -   第i种长度钢管的需求数量;
    - $l_i$ -   第i种长度钢管的长度;
    - $L_k$ -   第k根未切割钢管的长度。
- 变量
    - $x_{ik}$ - 表示第k根钢管切割第i种长度的数量;
    - $y_k$ - 表示第k根钢管是否使用
    
- obj
    - $min \ \sum_k y_k$
- s.t.
    - $\sum_k x_{ik} \ge D_i$, $\forall i \in i$ (满足所需钢管数量)
    - $\sum_i l_i x_{ik} \le L_k y_k$, $\forall k \in K$ (钢管切割长度不能超过本身长度)
    - $x_{ik} \in N$, $y_k \in \{0,1\}$, $\forall i \in I$, $k \in K$

In [119]:
# 这种方法非常的消耗资源，因为N的数量需要事先编一个
# 如果模型无解(即当前可用钢管数量不够), 则还需要增加N的数量
# 并且如果N设的太大也会消耗资源,效率低下


# import env
import gurobipy as gp
from gurobipy import GRB

# 假设当前钢管数量(若模型无解则需要增加N的数量，直至其有解)
N = 100

# Create Params
I, D, l = gp.multidict({
    '3m': [25, 3],
    '7m': [30, 7],
    '9m': [14, 9],
    '16m': [8, 16]
})

K, L = gp.multidict({i:20 for i in range(N)})

# Create Model
m = gp.Model('class4_example1')

# Create Vars
x = m.addVars(I, N, vtype=GRB.INTEGER, name='x')
y = m.addVars(N, vtype=GRB.BINARY, name='y')

# Set Objective
m.setObjective(
    gp.quicksum(y[k] for k in range(N))
)

# Create Constrs
m.addConstrs(
    (gp.quicksum(x[i,k] for k in range(N))
     == [D[i], GRB.INFINITY]
     for i in I),
    'c0'
)

# 这里需要注意，要把 (- L[k] * y[k]) 写到gp.quicksum()外，写到里面相当于会计算I次 (- L[k] * y[k])
m.addConstrs(
    (gp.quicksum(l[i] * x[i, k] for i in I) - L[k] * y[k]
     == [-GRB.INFINITY, 0]
     for k in range(N)),
    'c1'
)

# optimize
m.optimize()

# Print result
# print('The number of stock: {}'.format(m.objVal))

# Print result
for v in m.getVars():
    print('{} {}'.format(v.varName, v.x))
print('Obj: {}'.format(m.objVal))


Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 104 rows, 500 columns and 900 nonzeros
Model fingerprint: 0xb831bf46
Variable types: 0 continuous, 500 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 3e+01]
Found heuristic solution: objective 35.0000000
Presolve time: 0.00s
Presolved: 104 rows, 500 columns, 900 nonzeros
Variable types: 0 continuous, 500 integer (200 binary)

Root relaxation: objective 2.735000e+01, 205 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.35000    0   34   35.00000   27.35000  21.9%     -    0s
H    0     0                      33.0000000   27.35000  17.1%     -    0s
H    0     0                      30.0000000   27.35000  8.83%     -    0s
     0     0   27.350

**列生成思路**
- 参数
    - $C_{ip}$ - 表示第p种切割模式下，切出第i种长度的钢管数量
    - $P$ - 表示切割模式集合
    - $D_i$ -   第i种长度钢管的需求数量;
- 变量
    - $z_p$ - 表示第p种切割模式使用的次数
    
- obj
    - $min \ \sum_p z_p$
- s.t.
    - $\sum_p C_{ip} z_p \ge D_i$, $\forall i\in I$
    - $z_p \ Integer$, $\forall p \in P$
    
切割模式组合非常多，因此枚举所有$z_p$几乎是不可行也没有必要的。因为并不是所有的切割模式都会用到。例如，每根钢管只切割一种长度浪费较多。

**所以问题就到了如何寻找好的切割模式？**

- 列生成算法求解步骤
    1. 给定几种初始的切割模式$\bar{P} \in P$ 带入原列生成模型中，并将变量松弛为连续型
        - obj
            - $min \ \sum_p z_p$
        - s.t.
            - $\sum_p C_{ip} z_p \ge D_i$, $\forall i \in I$
            - $z_p \ge 0$, $\forall p \in \bar{P}$
        - 求解上述模型获得对应的对偶变量取值, 记为$\lambda_i$
    2. 思考是否存在新的切割模式$p_{new}$加到模型进而降低目标函数值。假设存在，则必然会有变量$z_{p_{new}}$的 Reduced Cost = $1 - \sum_i \lambda_i C_{ip_{new}} \lt 0$
        - obj
            - $min(\sum_p z_p) + z_{p_{new}}$
        - s.t.
            - $(\sum_p C_{ip} z_p) + C_{ip_{new}} z_{p_{new}} \ge D_i$, $\forall i \in I$
            - $z_p \ge 0$, $\forall p \in \bar{P} \bigcup p_{new}$
        - 为了获取更优的目标值，往往会选择Reduced Cost最小的切割模式加入模型中。**所以问题就到了如何确定Reduced Cost最小的切割模式**
    3. 通过一个子问题，确定Reduced Cost最小的切割模式。对于任意的切割模式，都需要满足钢管长度限制。因此可以构造如下模型，其中变量$C_i$表示该切割模式切除第i种长度的钢管数量
        - obj
            - $min \ 1 - \sum_i \lambda_i C_i$
        - s.t.
            - $\sum_i C_i l_i \le l$, $\forall i \in I$
            - $C_i \ Integer$, $\forall i \in I$
        - 如果子问题最优目标值小于0，那么意味着发现更好的切割模式，将新切割模式加入到模型中，重复以上步骤，直到没有更好的切割模式出现。

In [94]:
# 例子
# 给出四种初始切割模式
# 1. 3m切6块
# 2. 7m切2块
# 3. 9m切2块
# 4. 16m切1块
# 对于上述切割模式可以规划求出对偶值[0.16666, 0.5, 0.5, 1.0]
# 然后使用步骤3，发现Reduced Cost最小值为 -0.3333 < 0, 因此发现更好的切割模式
# 3m切2块, 7m切2块
# 将新的切割模式加入到模型中，继续迭代

In [101]:
# import env
import gurobipy as gp
from gurobipy import GRB

# Create Params
TypesDemand = [3, 7, 9, 16]       # 需求长度
QuantityDemand = [25, 30, 14, 8]  # 需求的量
LengthUsable = 20                 # 钢管长度

try:
    MainProbRelax = gp.Model()       # 松弛后的列生成主问题
    SubProb = gp.Model()             # 子问题
    
    # 构造主问题模型, 选择初始切割方案, 每根钢管只切一种长度
    # 添加变量
    Zp = MainProbRelax.addVars(
        len(TypesDemand), obj=1.0, vtype=GRB.CONTINUOUS, name = 'z'
    )
    # 添加约束
    ColumnIndex = MainProbRelax.addConstrs(
        gp.quicksum(Zp[p] * (LengthUsable//TypesDemand[i]) for p in range(len(TypesDemand)) if p==i) 
        >= QuantityDemand[i] 
        for i in range(len(TypesDemand))
    )
    # 求解
    MainProbRelax.optimize()
    
    
    # 构造子问题模型
    # 获得对偶值
    DualSolution = MainProbRelax.getAttr(GRB.Attr.Pi, MainProbRelax.getConstrs())
    # 添加变量
    Ci = SubProb.addVars(
        len(TypesDemand), obj=DualSolution, vtype=GRB.INTEGER, name='c'
    )
    # 添加约束
    SubProb.addConstr(
        gp.quicksum(Ci[i] * TypesDemand[i] for i in range(len(TypesDemand)))
        <= LengthUsable
    )
    # 设定优化方向为求最小
    SubProb.setAttr(GRB.Attr.ModelSense, -1)  
    # 求解
    SubProb.optimize()
    
    
    # 判断Reduced Cost是否小于0
    while SubProb.objval > 1:
        # 获取变量取值
        columnCoeff = SubProb.getAttr("X", SubProb.getVars())
        column = gp.Column(columnCoeff, MainProbRelax.getConstrs())
        # 添加变量
        MainProbRelax.addVar(obj=1.0, vtype=GRB.CONTINUOUS, name="CG", column=column)
        # 求解
        MainProbRelax.optimize()   
        # 修改子问题目标函数系数
        for i in range(len(TypesDemand)):
            Ci[i].obj = ColumnIndex[i].pi
        SubProb.optimize()
    
    # 将CG后的模型转为整数并求解
    for v in MainProbRelax.getVars():
        v.setAttr('VType', GRB.INTEGER)
    # 求解
    MainProbRelax.optimize()
    # 输出结果
    for v in MainProbRelax.getVars():
        if v.X != 0.0:
            print('{} {}'.format(v.VarName, v.X))
            
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')   

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xe4b82c3d
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 3e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4166667e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  3.416666667e+01
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x8c72f86d
Variable types: 0 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e+00, 2e+01]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 1.0000000
P

## Benders Decomposition

1. 
    - obj
        - $min \ c^Tx + f^T y$
    - s.t.
        - $Ax + By = b$
        - $x \ge 0$, $y \in Y$

假设变量y是一个比较复杂的变量，如果该变量的值可以提前确定，剩下的模型往往就比较容易求解。因此通过变量的区分，可以将元模型拆解为两个相对较小的模型，然后通过间接迭代求解小模型到达求解原模型的目的。

我们将上式拆解成如下两个规划:
2. 
    - obj
        - $min \ f^Ty + g(y)$
    - s.t.
        - $t \in Y$


3. 
    - $g(\bar{y}) = min \ c^T x$
    - $Ax = b - B \bar{y}$
    - $x \ge 0$

若模型3可以取到无穷小(无界) $\leftrightarrow$ 模型2可以取到无穷小 $\leftrightarrow$ 模型1可以取到无穷小

若模型3有界, 写出模型3的对偶模型
4. 
    - obj
        - $max \ \lambda^T (b - B \bar{y})$
    - s.t.
        - $A^T \lambda \le c$
        - $\lambda \ free$

可以看出，模型4的可行域与变量y无关，变量y的值只会影响其目标函数值。如果模型4的可行域为空集，那么根据对偶性可知模型3无界或者为空

假设模型4非空，可以枚举出所有的extreme points$(\alpha^1_p, \alpha^2_p, ..., \alpha^l_p)$ 和 extreme rays$(\alpha^1_r, \alpha^2_r, ..., \alpha^l_r)$。 因此，求解模型4可以等价为:
- 最优: 寻找$\alpha^i_p$使得$\alpha_p^{i^{T}} (b - B \bar{y})$最大化
- 排除不可行: 检测$\alpha_r^{j^{T}} (b - B \bar{y}) \le 0$是否成立。如果$\alpha_r^{j^{T}} (b - B \bar{y}) \gt 0$，模型4无界，进而模型3不可行。

因此模型4可以从:
- $max_{i \in \{1,2,...,l\}} \alpha_p^{i^{T}} (b - B \bar{y})$
- $\alpha_r^{j^{T}} (b - B \bar{y}) \le 0$, $\forall j \in \{1,2,...,J\}$

转化为
5. 
    - obj
        - $min \ z$
    - s.t.
        - $\alpha_p^{i^{T}} (b - B \bar{y}) \le z$, $\forall i \in \{1,2,...,I\}$
        - $\alpha_r^{j^{T}} (b - B \bar{y}) \le 0$, $\forall j \in \{1,2,...,J\}$
        - $z \ free$

将模型5带入到模型2中，可以得到模型1的一个等价模型6
6. 
    - obj
        - $min \ f^Ty + z$
    - s.t.
        - $\alpha_p^{i^{T}} (b - B \bar{y}) \le z$, $\forall i \in \{1,2,...,I\}$
        - $\alpha_r^{j^{T}} (b - B \bar{y}) \le 0$, $\forall j\in \{1,2,...,J\}$
        - $y \in Y$, $z free$

模型6包含变量y和z，以及大量的约束，这些约束在实际求解是很难全部美剧的，因此很难直接求解模型6

因此Benders Decomposition的思路是不直接求解模型6， 具体迭代思路如下：

首先, 找到一组候选解$(y^*, z^*)$, 将$y^*$带入到模型3中，获得如下模型(称之为Benders subproblem, SP):

- $g(y^*) = min \ c^T x$
- $Ax = b - B y^*$
- $x \ge 0$

求解SP模型可能有三种结果:
- 最优解且$g(y^*) = z^*$, 发现原问题最优解, 停止迭代
- 最优解且$g(y^*) \gt z^*$, 可以构造一个形如$\alpha_p^T (b - B y) \le z$(Benders Optimality Cut) 加入到Benders Master Problem(MP)
- 不可行, 可以构造一个形如$\alpha_r^T (b - B y) \le 0$ (Benders Fesibility Cut)加入到 Benders master problem, MP

进而得到一个随着迭代规模不断增加的Benders Master Problem Model, MP
- obj
    - $min \ f^T y + z$
- s.t.
    - $\alpha_p^T (b - B y) \le z$. (Benders Optimality Cut) (MP)
    - $\alpha_r^T (b - B y) \le 0$. (Benders Feasibility Cut)
    - $y \in Y$, $z \ free$

接着求解MP, 获得一组新的$(y^*, z^*)$, 然后重复上述迭代过程直到$g(y^*) = z^*$

In [121]:
# 总结
# Identify a MP and an easy SP(y)
# repeat
#     solve MP obtaining the solution (y*, z*)
#     solve SP(y*)
#     if SP(y*) is feasible:
#         if obj(SP(y*)) == z*:
#             STOP
#         else:
#             add to MP the Benders Optimality Cut
#     else:
#         add to MP the Benders Feasibility Cut
# until(end condition)

**案例**
- $min \ x_1 + x_2 + ... + x_5 + 7y_1 + 7y_2 + ... + 7y_5$
- $x_1 + x_4 + x_5 = 8$
- $x_2 + x_5 = 3$
- $x_3 + x_4 = 5$
- $x_1 \le 8y_1$
- $x_2 \le 3y_2$
- $x_3 \le 5y_3$
- $x_4 \le 5y_4$
- $x_5 \le 3y_5$
- $x_1, x_2, ..., x_5 \ge 0$, $y_1, y_2, ..., y_5 \in \{0,1

首先确定合适的MP和SP, 针对本数值案例, 变量x,y 分别为continuous和binary, 所以变量y相对复杂, 因此构建MP模型如下:
- $min \ 7y_1 + 7y_2 + ... + 7y_5 + z$
- $z \ge 0$, $y_1, y_2, ..., y_5 \in \{0,1\}$

求解该MP模型, 获得最优解$y^*=(0,0,0,0,0)$, $z^ = 0$。将该候选解带入SP模型:
- $min \ x_1 + x_2 + ... + x_5 + 7*0 + 7*0 + ... + 7*0$
- $x_1 + x_4 + x_5 = 8$
- $x_2 + x_5 = 3$
- $x_3 + x_4 = 5$
- $x_1 \le 8*0$
- $x_2 \le 3*0$
- $x_3 \le 5*0$
- $x_4 \le 5*0$
- $x_5 \le 3*0$
- $x_1, x_2, ..., x_5 \ge 0$

虽然SP模型不可解，Dual-SP无界, 可以获得Dual-SP的一个extreme ray(1, 0, 0, -1, 0, 0, -1, -1)可以向MP模型中添加Benders Feasibility Cut
- $8y_1 + 5y_4 + 3y_5 \ge 8$

更新MP模型
- $min \ 7y_1 + 7y_2 + ... + 7y_5 + z$
- $8y_1 + 5y_4 + 3y_5 \ge 8$
- $z \ge 0$, $y_1, y_2, ..., y_5 \in \{0,1\}$

求解该MP模型获得最优解$y^* = (1,0,0,0,0)$, $z^*=0$。将该候选解带入SP模型:
- $min \ x_1 + x_2 + ... + x_5 + 7*1 + 7*0 + ... + 7*0$
- $x_1 + x_4 + x_5 = 8$
- $x_2 + x_5 = 3$
- $x_3 + x_4 = 5$
- $x_1 \le 8*1$
- $x_2 \le 3*0$
- $x_3 \le 5*0$
- $x_4 \le 5*0$
- $x_5 \le 3*0$
- $x_1, x_2, ..., x_5 \ge 0$

虽然SP模型仍不可解，Dual-SP无界, 可以获得Dual-SP的一个extreme ray(0,1,1,0,-1,-1,-1,-1),再可以向MP模型中添加Benders Feasibility Cut
- $3y_2 + 5y_3 + 5y_4 + 3y_5 \ge 8$

继续更新MP模型
- $min \ 7y_1 + 7y_2 + ... + 7y_5 + z$
- $8y_1 + 5y_4 + 3y_5 \ge 8$
- $3y_2 + 5y_3 + 5y_4 + 3y_5 \ge 8$
- $z \ge 0$, $y_1, y_2, ..., y_5 \in \{0,1\}$

求解该MP模型获得最优解$y^* = (0,0,0,1,1)$, $z^*=0$。将该候选解带入SP模型:
- $min \ x_1 + x_2 + ... + x_5 + 7*0 + 7*0 + ... + 7*1$
- $x_1 + x_4 + x_5 = 8$
- $x_2 + x_5 = 3$
- $x_3 + x_4 = 5$
- $x_1 \le 8*0$
- $x_2 \le 3*0$
- $x_3 \le 5*0$
- $x_4 \le 5*1$
- $x_5 \le 3*1$
- $x_1, x_2, ..., x_5 \ge 0$

SP模型有最优解 obj = 22 > 0, 因此可以向MP模型添加Benders Optimality Cut
- $5y_4 + 3y_5 + z \ge 16$

继续更新MP模型
- $min \ 7y_1 + 7y_2 + ... + 7y_5 + z$
- $8y_1 + 5y_4 + 3y_5 \ge 8$
- $3y_2 + 5y_3 + 5y_4 + 3y_5 \ge 8$
- $5y_4 + 3y_5 + z \ge 16$
- $z \ge 0$, $y_1, y_2, ..., y_5 \in \{0,1\}$

求解该MP模型获得最优解$y^* = (0,0,0,1,1)$, $z^*=8$。因为$y^*$与上一步值相同，因此带入SP模型最优解obj = 22。发现了原问题的最优解，停止迭代过程。

获得原问题最优解obj = 22, x = (0, 0, 0, 5, 3), y = (0, 0, 0, 1, 1)

In [122]:
from gurobipy import *
def addBendersCuts(SP_Dual_obj, x):
    if SP_Dual.status == GRB.Status.UNBOUNDED:       #模型无界
        ray = SP_Dual.UnbdRay
        MP.addConstr(8*ray[0] + 3*ray[1] + 5*ray[2] + \
                     8*ray[3]*y[0] + 3*ray[4]*y[1] + 5*ray[5]*y[2] + \
                     5*ray[6]*y[3] + 3*ray[7]*y[4]<= 0)
    elif SP_Dual.status == GRB.Status.OPTIMAL:       #发现最优解
        MP.addConstr(8*Vdual1[0].x + 3*Vdual1[1].x + 5*Vdual1[2].x + \
                     8*Vdual2[0].x*y[0] + 3*Vdual2[1].x*y[1] + 5*Vdual2[2].x*y[2] + \
                     5*Vdual2[3].x*y[3] + 3*Vdual2[4].x*y[4] <= z)
        SP_Dual_obj[0] = SP_Dual.ObjVal              #获取最优解
        x.append(x1.pi)                              #获取SP模型解
        x.append(x2.pi)
        x.append(x3.pi)
        x.append(x4.pi)
        x.append(x5.pi)
    else:                                            #其他状态
        print (SP_Dual.status)
        

try:
    MP = Model()            #Benders Master Problem
    SP_Dual = Model()       #dual of Benders SubProblem
    
    #add Variables
    y= MP.addVars(5, obj=7, vtype=GRB.BINARY, name='y')
    z= MP.addVar(obj=1, vtype=GRB.CONTINUOUS, name='z')
    Vdual1 = SP_Dual.addVars(3, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='Vdual1')
    Vdual2 = SP_Dual.addVars(5, lb=-GRB.INFINITY,ub=0, vtype=GRB.CONTINUOUS, name='Vdual2')
    
    x1 = SP_Dual.addConstr(Vdual1[0] + Vdual2[0] <=1)
    x2 = SP_Dual.addConstr(Vdual1[1] + Vdual2[1] <=1)
    x3 = SP_Dual.addConstr(Vdual1[2] + Vdual2[2] <=1)
    x4 = SP_Dual.addConstr(Vdual1[0] + Vdual1[2] + Vdual2[3] <=1)
    x5 = SP_Dual.addConstr(Vdual1[0] + Vdual1[1] + Vdual2[4] <=1)
    
    #设置参数 InfUnbdInfo
    SP_Dual.Params.InfUnbdInfo = 1
    
    iteration = 0
    SP_Dual_obj = [9999]
    x = []
    MP.optimize()

    while SP_Dual_obj[0] > z.x:
        if iteration == 0: 
            SP_Dual.setObjective(8*Vdual1[0] + 3*Vdual1[1] + 5*Vdual1[2] + \
                                  8*Vdual2[0]*y[0].x + 3*Vdual2[1]*y[1].x + 5*Vdual2[2]*y[2].x + \
                                  5*Vdual2[3]*y[3].x + 3*Vdual2[4]*y[4].x, GRB.MAXIMIZE)
            SP_Dual.optimize()
            addBendersCuts(SP_Dual_obj, x)
            iteration = 1
        else:
            Vdual2[0].obj = 8*y[0].x
            Vdual2[1].obj = 3*y[1].x
            Vdual2[2].obj = 5*y[2].x
            Vdual2[3].obj = 5*y[3].x
            Vdual2[4].obj = 3*y[4].x
            SP_Dual.optimize()
            addBendersCuts(SP_Dual_obj, x)
            iteration = iteration + 1
        
        MP.optimize()
      
    for i in range(5):
        print('x[%d] = %f'%(i, x[i]))
        
    for i in range(5):
        print('y[%d] = %d'%(i, y[i].x))
     
except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')        


Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 0 rows, 6 columns and 0 nonzeros
Model fingerprint: 0xe77649a4
Variable types: 1 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 5 rows, 8 columns and 12 nonzeros
Model fingerprint: 0xfb8f5e29
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  R